In [1]:
import datetime as dt
import pandas as pd
import numpy as np
from scipy.stats import norm

In [ ]:
def calculate_delta(S, K, T, t, r, sigma, eps):
    d1 = (np.log(S/K) + (r + sigma ** 2 / 2) * (T - t)) / (sigma * np.sqrt(T - t) + eps)
    return norm.cdf(d1)

In [ ]:
# Black-Scholes Call Price
def black_scholes_call(S, K, T, t, r, sigma, eps):
    d1 = (np.log(S/K) + (r + sigma ** 2 / 2) * (T - t)) / (sigma * np.sqrt(T - t) + eps)
    d2 = d1 - sigma * np.sqrt(T)
    return S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)

In [ ]:
def interval_func(start, end, interval):
    result = []
    current = start
    while current <= end:
        result.append(round(current, 4))
        current += interval
    return result

In [ ]:
def calc_delta(S, K, T, t, r, C_0, eps=np.finfo(float).eps, tol=0.2):

    for sigma in interval_func(0, 4, 0.001):
        c = black_scholes_call(S=S, K=K, T=T, t=t, r=r, sigma=sigma, eps=eps)
        if abs(c-C_0) <= tol:
            delta = calculate_delta(S=S, K=K, T=T, t=t, r=r, sigma=sigma, eps=eps)
            print(f"Delta = {delta}, sigma = {sigma} call price from BS = {round(c, 5)}")
            return delta
    print(c)
    return 0.0

In [ ]:
# Alkuarvot
# Data:
data = pd.read_feather('firma1.feather')

# drop unnecessary three rows as our hedging starting day is 2024-09-10
df = data.drop(index=[0,1,2]).reset_index(drop=True)

start_date = dt.datetime.strptime("2024-09-10", '%Y-%m-%d')
maturity_date = dt.datetime.strptime('2024-10-25', '%Y-%m-%d')

# ATM 2024-09-10
T = 45/365
t = 0
K = 105

strike = f'C{K}'
C_0 = df[strike].iloc[0]
S_0 = df['Underlying'].iloc[0]

# Risk-free rate is us 30-day treasure bond risk-free rate at 2024-09-10
r = 0.0497
eps = np.finfo(float).eps
delta = calc_delta(S=S_0, K=K, T=T, t=t, r=r, C_0=C_0, eps=np.finfo(float).eps, tol=0.1)
print(f"Delta at S_O= {delta}")

# Hedging interval
interval = 2

In [ ]:
"""
We want to re-hedge the portfolio at specific intervals and calculate mean square error E = (1 / n - 1) * SUM_i=1->n-1(A^2)
We choose to hedge every second day.
Function calculates both OP and RE portfolio values and their difference A_i as the result.
Re-hedging is done by calculating new delta values.

OP = c_i+1 - c_1
RE = delta_i(s_i+1 - s_i)
A_i = OP + RE
E = (1 / n - 1) * SUM_i=1->n-1(A_i^2)

interval = re-hedging interval (days)
strike = strike price in format ('C{strike_price}) e.g. 'C105'
"""

OP_0 = C_0

# delta * S_0
RE_0 = delta * S_0



def hedging(interval, strike, df):
    A_boss = 0
    interval_count = 1
    c_0 = C_0
    c_1 = 0
    s_0 = S_0
    s_1 = 0
    delta = delta
    t = 0
    n = 0

    for _, row in df.iterrows():
        c_1 = float(row[strike])
        s_1 = float(row['Underlying'])
        t = (45 - (maturity_date - row['Date']).days) / 365
        
        if row.name == 1:
            continue

        OP = c_1 - c_0

        
        RE = delta * (s_1-s_0)
        A = OP + RE
        A_boss += A ** 2
        n += 1
        
        if interval_count % interval == 0:
          print(row['Date'].strftime("%Y-%m-%d"))
          delta = calc_delta(s_1, K, T, t, r, c_1)
          

        c_0 = c_1
        s_0 = s_1
        interval_count += 1

    mse = A_boss/(1-n)
    
    return mse



In [ ]:
hedging(interval=interval, strike=strike, df=df)